# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

Theoretically, if this works on A100, it should work on Zach's Quadros since they both have around 40 GB vRAM.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
import os

login(token=os.environ['HUGGINGFACE_WRITE_KEY'])
wandb_key = os.environ['WANDB_KEY']
# for convenience you can create an environment variable containing your hub token as HF_TOKEN

KeyError: 'WANDB_KEY'

In [ ]:
# !pip install trl

In [4]:
from peft import LoraConfig

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "google/gemma-2-2b"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)


In [6]:
dataset_path = "HuggingFaceTB/smoltalk"
# Set our name for the finetune to be saved &/ uploaded to
finetune_name = f"{model_name.split('/')[1]}-FT-{dataset_path}"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [ ]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
del inputs, outputs


In [8]:
torch.cuda.empty_cache()

## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [9]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path=dataset_path, name="everyday-conversations")

In [10]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [11]:
output_dir = '/content/drive/MyDrive/CS/Fall24/Zach-Wood-Doughty_CS_Research_Track/Interpretable Fine Tuning/training'

In [ ]:
# prompt: Print the model's named modules

import torch

print(list(model.named_modules()))


In [ ]:
from peft import LoraConfig, TaskType, get_peft_model
from transformers import TrainingArguments

target_modules = [
    "model.layers.25.self_attn.q_proj",
    "model.layers.25.self_attn.k_proj",
    "model.layers.25.self_attn.v_proj",
    "model.layers.25.self_attn.o_proj",
    "model.layers.25.mlp.gate_proj",
    "model.layers.25.mlp.up_proj",
    "model.layers.25.mlp.down_proj"
]

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=target_modules,
    bias="none",
)

# Convert model to fp16 before applying LoRA
model = model.half()

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Configure the training arguments with mixed precision training
training_args = TrainingArguments(
    output_dir=output_dir+"/sft_output",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=32,
    num_train_epochs=3,
    weight_decay=0.01,
    learning_rate=5e-5,
    logging_steps=10,
    save_strategy="epoch",
    eval_steps=50,
    use_mps_device=True if device == "mps" else False,
    hub_model_id=finetune_name,
    save_total_limit=2,
    load_best_model_at_end=True,
    # Configure mixed precision training
    fp16=True,
    fp16_full_eval=True,  # Use fp16 for evaluation as well
    half_precision_backend="auto",
    hub_model_id = "AMindToThink/gemma-2-2b-FT-HuggingFaceTB",

)

# Initialize the SFTTrainer with LoRA-enabled model
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# Train the model
trainer.train()

In [ ]:
# # Configure the SFTTrainer
# sft_config = SFTConfig(
#     output_dir=output_dir+"/sft_output",
#     evaluation_strategy="epoch",
#     # max_steps=1000,  # Adjust based on dataset size and desired training duration
#     per_device_train_batch_size=1,  # Set according to your GPU memory capacity
#     gradient_accumulation_steps=32,
#     num_train_epochs=3,
#     weight_decay=0.01,

#     learning_rate=5e-5,  # Common starting point for fine-tuning
#     logging_steps=10,  # Frequency of logging training metrics
#     save_strategy="epoch",
#     # save_steps=100,  # Frequency of saving model checkpoints
#     # evaluation_strategy="steps",  # Evaluate the model at regular intervals
#     eval_steps=50,  # Frequency of evaluation
#     use_mps_device=(
#         True if device == "mps" else False
#     ),  # Use MPS for mixed precision training
#     hub_model_id=finetune_name,  # Set a unique name for your model
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     fp16=True,
# )

# # Initialize the SFTTrainer
# trainer = SFTTrainer(
#     model=model,
#     args=sft_config,
#     train_dataset=ds["train"],
#     tokenizer=tokenizer,
#     eval_dataset=ds["test"],
# )

# # TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# prompt: Init wandb

import wandb
wandb.login(key=wandb_key)
wandb.init(project="interp_fine_tuning")


In [ ]:
# Train the model

trainer.train()


In [22]:

# Save the model
trainer.save_model(f"{output_dir}/{finetune_name}")

In [26]:
login(token=userdata.get("HF_WRITE_TOKEN"))

In [58]:
trainer.args.hub_model_id = "MKhoriaty/gemma-2-2b-FT-HuggingFaceTB"


In [48]:
# !huggingface-cli logout
from huggingface_hub import logout
import os
os.environ['HF_TOKEN'] = os.environ['HUGGING_FACE_HUB_TOKEN'] = "None"

In [ ]:
login(userdata.get('HF_WRITE_TOKEN'))

In [ ]:
userdata.get("HF_TOKEN")

In [ ]:
# First, let's clear any existing token
from huggingface_hub import logout
logout()

# Remove token from environment variable if it exists
import os
if 'HUGGING_FACE_HUB_TOKEN' in os.environ:
    del os.environ['HUGGING_FACE_HUB_TOKEN']

# Now login with your token
from huggingface_hub import login
login(token="your_token_here", add_to_git_credential=True)

# Verify the login
from huggingface_hub import whoami
print("Logged in as:", whoami())

In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import whoami
current_user = whoami()
print(current_user)

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [60]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

In [ ]:
# prompt: Make a text generation pipe from the best model and the tokenizer

from transformers import pipeline

# Replace with your actual model and tokenizer
model_id = "MKhoriaty/gemma-2-2b-FT-HuggingFaceTB"
pipe = pipeline("text-generation", model=model_id, tokenizer=model_id, device=0)


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.